In [ ]:
import sys
import os
from pathlib import Path
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import re
import ipywidgets as widgets
from IPython.display import display
# The NEAT-Python library imports
import modneat
from modneat import visualize
# import task
current_dir = str(Path().resolve())
sys.path.append(os.path.join(current_dir, '..'))
import settings.task as task

#load experiment settings
experiment_settings = modneat.report_funcs.load_experiment_settings(path_of_command_file="../settings/command")

#load checkpoint
checkpoints_path = '../checkpoints/'
checkpoints = os.listdir(checkpoints_path)
sorted_checkpoints = sorted(checkpoints, key=lambda x: int(x.split('-')[1]))

# 選択メニュー用のウィジェットを作成
dropdown = widgets.Dropdown(
    options=sorted_checkpoints,  # リストから選択肢を指定
    description='Select a checkpoint:',
    disabled=False,
)

global selected_checkpoint
selected_checkpoint = sorted_checkpoints[0]

# 選択メニューの値が変更されたときに実行される関数
def on_change(change):
    global selected_checkpoint
    if change['type'] == 'change' and change['name'] == 'value':
        selected_checkpoint = change['new']
        print(f"Selected checkpoint: {selected_checkpoint}")

# 選択メニューの値が変更されたときに on_change 関数を呼び出す
dropdown.observe(on_change)

# ウィジェットを表示
display(dropdown)
print(f"Selected checkpoint: {selected_checkpoint}")

In [ ]:
# checkpointo, taskをロード
print(selected_checkpoint)
p = modneat.Checkpointer.restore_checkpoint(os.path.join('../checkpoints', selected_checkpoint))
task_name = experiment_settings['task']
network_name = experiment_settings['network']
task_call_string = 'task.' + task_name + '(network_type = modneat.nn.' + network_name + ')'
print(">>> ",task_call_string)
t = eval(task_call_string)

# タスクを実行し、ベスト個体を取得
best_genome = p.run(t.eval_genomes, 1)

# ベスト個体のネットワークを作成
net_call_string = 'modneat.nn.' + network_name + '.create(best_genome, p.config)'
print(">>> ",net_call_string)
net = eval(net_call_string)

fitness, log = t.eval_fitness(net)
print("#fitness: ", fitness)
# log を利用してnetworkの挙動を可視化していく
print(log)

In [ ]:
modneat.visualize.draw_net(p.config, best_genome, show_disabled=False)

In [ ]:
#各ノードの活性化値の可視化

cols = ['step', 'node_id', 'values']
df = pd.DataFrame(index=[], columns = cols)
hist = [items['values'] for items in log]

step = 0
for values in hist:
    for i in values.items():
        record = pd.Series([step, i[0], i[1]], index=df.columns)
        df = df.append(record, ignore_index=True)
    step += 1

df['step'] = df['step'].astype(int)
df['node_id'] = df['node_id'].astype(int)
df_pivot = pd.pivot_table(data = df, values='values', columns='node_id', index='step')
x_num = len(df_pivot.columns)
y_num = len(df_pivot.values)
sns.set(rc={'figure.figsize':(int(x_num*2), int(y_num*0.3))})
sns.heatmap(df_pivot, annot=True, fmt='.2f', linewidth=.5)

In [ ]:
#各ノードの修飾値の可視化

cols = ['step', 'node_id', 'mod_values']
df = pd.DataFrame(index=[], columns = cols)
hist = [items['modulate_values'] for items in log]

step = 0
for values in hist:
    for i in values.items():
        record = pd.Series([step, i[0], i[1]], index=df.columns)
        df = df.append(record, ignore_index=True)
    step += 1

df['step'] = df['step'].astype(int)
df['node_id'] = df['node_id'].astype(int)

df_pivot = pd.pivot_table(data = df, values='mod_values', columns='node_id', index='step')
x_num = len(df_pivot.columns)
y_num = len(df_pivot.values)
sns.set(rc={'figure.figsize':(int(x_num*2), int(y_num*0.3))})
sns.heatmap(df_pivot, annot=True, fmt='.2f', linewidth=.5)

In [ ]:
### 重みの変化を表示

cols = ['step', 'connection', 'weight']
df = pd.DataFrame(index=[], columns = cols)
node_evals = [items['node_evals'] for items in log]

print(node_evals[0][1])

step = 0
for node_eval in node_evals:
    
    for n in range(len(node_eval)):
    
        for each_connection_info in node_eval[n][-1]:
            record = pd.Series([step, str(each_connection_info[0]) + ' to ' + str(node_eval[n][0]), each_connection_info[1]], index=df.columns)
            df = df.append(record, ignore_index=True)
    step += 1

df['step'] = df['step'].astype(int)
#df['connection'] = df['connection'].astype(int)

df_pivot = pd.pivot_table(data = df, values='weight', columns='connection', index='step')
x_num = len(df_pivot.columns)
y_num = len(df_pivot.values)
sns.set(rc={'figure.figsize':(int(x_num*2.2), int(y_num*0.3))})
sns.heatmap(df_pivot, annot=True, fmt='.2f', linewidth=.5)